In [16]:
import numpy as np
import scipy.stats as stats
import pandas as pd

data = pd.read_csv("train.csv")

#Entferne alle von H als Outlier markierte Zeilen
data = data.drop(data[data["H"] == 1].index)
data = data.drop("H", axis="columns")

#Entferne weitere Outlier aus den restlichen Spalten
Q1 = data.quantile(q=.25)
Q3 = data.quantile(q=.75)

IQR = data.apply(stats.iqr)

data_clean = data[~((data < (Q1-1.5*IQR)) | (data > (Q3+1.5*IQR))).any(axis=1)]

data_clean

#Dimensionsreduktion, PCA/Feature Selection?
#Unser Target ist 0 oder 1, denk mal logistische Regression macht am meisten Sinn?


,A1,B1,C1,C3,D1,E1,F,G,I,D2,E2,A2,B2,C2,Target
0,1,0,0,0,0,0,35.06,1.07,46.28,0.0,0.0,0.22,0.0,0.0,0
1,1,0,0,0,0,0,40.40,0.73,91.22,0.0,0.0,0.11,0.0,0.0,0
3,1,0,0,0,0,0,51.95,7.66,3.27,0.0,0.0,3.05,0.0,0.0,0
6,1,0,0,0,0,0,34.33,1.47,1.21,0.0,0.0,8.27,0.0,0.0,0
7,1,0,0,0,0,0,34.37,1.77,1.58,0.0,0.0,6.32,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41120,52,0,0,0,0,0,36.02,4.84,61.79,0.0,0.0,8.42,0.0,0.0,0
41122,25,0,0,0,0,0,41.04,6.67,33.78,0.0,0.0,7.40,0.0,0.0,0
41123,1,0,0,0,0,0,35.52,0.61,63.27,0.0,0.0,0.16,0.0,0.0,0
41124,1,0,0,0,0,0,51.37,2.09,65.05,0.0,0.0,0.15,0.0,0.0,0
